Let's make sure that we have access to GPU. We can use `nvidia-smi` command to do that. In case of any problems navigate to `Edit` -> `Notebook settings` -> `Hardware accelerator` and set it to `GPU`.

In [ ]:
!nvidia-smi


## Install Detectron2 and dependencies

In [ ]:
!pip install git+https://github.com/facebookresearch/detectron2.git

Now is a good time to confirm that we have the right versions of the libraries at our disposal.

In [ ]:
!pip install -U "iopath<0.1.10,>=0.1.7"
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
import pkg_resources
print("detectron2:", pkg_resources.get_distribution("detectron2").version)

In [ ]:
# COMMON LIBRARIES
import os
import cv2

from datetime import datetime
import cv2

# DATA SET PREPARATION AND LOADING
from detectron2.data.datasets import register_coco_instances
from detectron2.data import DatasetCatalog, MetadataCatalog

# VISUALIZATION
from detectron2.utils.visualizer import Visualizer
from detectron2.utils.visualizer import ColorMode

# CONFIGURATION
from detectron2 import model_zoo
from detectron2.config import get_cfg

# EVALUATION
from detectron2.engine import DefaultPredictor

# TRAINING
from detectron2.engine import DefaultTrainer

## Run a Pre-trained Detectron2 Model

In [ ]:
# !wget http://images.cocodataset.org/val2017/000000439715.jpg -q -O input.jpg
# image = cv2.imread("./input.jpg")
# cv2.imshow('Display Window', image)  # 'Display Window' is the name of the window
# cv2.waitKey(0)  # Waits for a key press to close the window
# cv2.destroyAllWindows()  # Closes the window

In [ ]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117 --upgrade --force-reinstall

## COCO Format Dataset

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="RovqaIFPpcekpUVWjRke")
project = rf.workspace("tank-5yib6").project("tank-and-ramp")
version = project.version(1)
dataset = version.download("coco-segmentation")


### Register

When you use Detectron2, before you actually train the model you need to [register it](https://detectron2.readthedocs.io/en/latest/tutorials/datasets.html#register-a-coco-format-dataset).

In [ ]:
DATA_SET_NAME = dataset.name.replace(" ", "-")
ANNOTATIONS_FILE_NAME = "_annotations.coco.json"

In [ ]:
# TRAIN SET
TRAIN_DATA_SET_NAME = f"{DATA_SET_NAME}-train"
TRAIN_DATA_SET_IMAGES_DIR_PATH = os.path.join(dataset.location, "train")
TRAIN_DATA_SET_ANN_FILE_PATH = os.path.join(dataset.location, "train", ANNOTATIONS_FILE_NAME)

register_coco_instances(
    name=TRAIN_DATA_SET_NAME,
    metadata={},
    json_file=TRAIN_DATA_SET_ANN_FILE_PATH,
    image_root=TRAIN_DATA_SET_IMAGES_DIR_PATH
)

# TEST SET
TEST_DATA_SET_NAME = f"{DATA_SET_NAME}-test"
TEST_DATA_SET_IMAGES_DIR_PATH = os.path.join(dataset.location, "test")
TEST_DATA_SET_ANN_FILE_PATH = os.path.join(dataset.location, "test", ANNOTATIONS_FILE_NAME)

register_coco_instances(
    name=TEST_DATA_SET_NAME,
    metadata={},
    json_file=TEST_DATA_SET_ANN_FILE_PATH,
    image_root=TEST_DATA_SET_IMAGES_DIR_PATH
)

# VALID SET
VALID_DATA_SET_NAME = f"{DATA_SET_NAME}-valid"
VALID_DATA_SET_IMAGES_DIR_PATH = os.path.join(dataset.location, "valid")
VALID_DATA_SET_ANN_FILE_PATH = os.path.join(dataset.location, "valid", ANNOTATIONS_FILE_NAME)

register_coco_instances(
    name=VALID_DATA_SET_NAME,
    metadata={},
    json_file=VALID_DATA_SET_ANN_FILE_PATH,
    image_root=VALID_DATA_SET_IMAGES_DIR_PATH
)

We can now confirm that our custom dataset was correctly registered using [MetadataCatalog](https://detectron2.readthedocs.io/en/latest/modules/data.html#detectron2.data.MetadataCatalog).

In [ ]:
[
    data_set
    for data_set
    in MetadataCatalog.list()
    if data_set.startswith(DATA_SET_NAME)
]

### Visualize

Let's take a look at single entry from out train dataset.

In [ ]:
metadata = MetadataCatalog.get(TRAIN_DATA_SET_NAME)
dataset_train = DatasetCatalog.get(TRAIN_DATA_SET_NAME)

dataset_entry = dataset_train[0]
image = cv2.imread(dataset_entry["file_name"])

visualizer = Visualizer(
    image[:, :, ::-1],
    metadata=metadata,
    scale=0.5,
    instance_mode=ColorMode.IMAGE_BW
)

out = visualizer.draw_dataset_dict(dataset_entry)
cv2.imshow("visualize",out.get_image()[:, :, ::-1])
cv2.waitKey(0)  # Waits for a key press to close the window
cv2.destroyAllWindows()  # Closes the window

## Train Model Using Custom COCO Format Dataset

### Configuration

In [ ]:
# HYPERPARAMETERS
ARCHITECTURE = "mask_rcnn_R_101_FPN_3x"
CONFIG_FILE_PATH = f"COCO-InstanceSegmentation/{ARCHITECTURE}.yaml"
MAX_ITER = 2000
EVAL_PERIOD = 200
BASE_LR = 0.001
NUM_CLASSES = 3

# OUTPUT DIRa
OUTPUT_DIR_PATH = os.path.join(
    DATA_SET_NAME,
    ARCHITECTURE,
    datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
)

os.makedirs(OUTPUT_DIR_PATH, exist_ok=True)

In [ ]:
# cfg = get_cfg()
# cfg.merge_from_file(model_zoo.get_config_file(CONFIG_FILE_PATH))
# cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(CONFIG_FILE_PATH)
# cfg.DATASETS.TRAIN = (TRAIN_DATA_SET_NAME,)
# cfg.DATASETS.TEST = (TEST_DATA_SET_NAME,)
# cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64
# cfg.TEST.EVAL_PERIOD = EVAL_PERIOD
# cfg.DATALOADER.NUM_WORKERS = 2
# cfg.SOLVER.IMS_PER_BATCH = 2
# cfg.INPUT.MASK_FORMAT='bitmask'
# cfg.SOLVER.BASE_LR = BASE_LR
# cfg.SOLVER.MAX_ITER = MAX_ITER
# cfg.MODEL.ROI_HEADS.NUM_CLASSES = NUM_CLASSES
# cfg.OUTPUT_DIR = OUTPUT_DIR_PATH

In [ ]:

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(CONFIG_FILE_PATH))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(CONFIG_FILE_PATH)
cfg.DATASETS.TRAIN = (TRAIN_DATA_SET_NAME,)
cfg.DATASETS.TEST = (TEST_DATA_SET_NAME,)
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512 # Default batch size per image

# Adjusting batch size and image size for GPU with 6GB VRAM
cfg.SOLVER.IMS_PER_BATCH = 3  # Overall batch size
cfg.INPUT.MIN_SIZE_TRAIN = (640, 672, 704, 736, 768, 800)  # Adjusted to fit memory constraints
cfg.INPUT.MIN_SIZE_TEST = 800
cfg.INPUT.MAX_SIZE_TRAIN = 1333
cfg.INPUT.MAX_SIZE_TEST = 1333

cfg.TEST.EVAL_PERIOD = EVAL_PERIOD
cfg.DATALOADER.NUM_WORKERS = 4
cfg.SOLVER.BASE_LR = BASE_LR
cfg.SOLVER.MAX_ITER = MAX_ITER
cfg.MODEL.ROI_HEADS.NUM_CLASSES = NUM_CLASSES
cfg.INPUT.MASK_FORMAT = 'bitmask'
cfg.OUTPUT_DIR = OUTPUT_DIR_PATH

# Save config for future reference
with open(os.path.join(OUTPUT_DIR_PATH, "config.yaml"), "w") as f:
    f.write(cfg.dump())

### Training

In [ ]:
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir $OUTPUT_DIR_PATH

### Evaluation

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7


predictor = DefaultPredictor(cfg)

In [ ]:
dataset_valid = DatasetCatalog.get(VALID_DATA_SET_NAME)

for d in dataset_valid:
    img = cv2.imread(d["file_name"])
    outputs = predictor(img)

    visualizer = Visualizer(

        img[:, :, ::-1],
        metadata=metadata,
        scale=0.8,
        instance_mode=ColorMode.IMAGE_BW
    )
    out = visualizer.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2.imshow("validation",out.get_image()[:, :, ::-1])
    cv2.waitKey(0)  # Waits for a key press to close the window
    cv2.destroyAllWindows()  # Closes the window

In [2]:
import os
import cv2
import torch
from PIL import Image
from torchvision import transforms
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

def load_model(config_file, weights_file):
    # Load the configuration
    cfg = get_cfg()
    cfg.merge_from_file(config_file)
    cfg.MODEL.WEIGHTS = weights_file
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.96
      # Set a custom testing threshold
    return DefaultPredictor(cfg)

def process_image(predictor, image_path):
    # Load image
    image = cv2.imread(image_path)
    outputs = predictor(image)
    
    # Visualize the results
    v = Visualizer(image[:, :, ::-1], MetadataCatalog.get(predictor.cfg.DATASETS.TRAIN[0]), scale=1.2)
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    
    # Display the image
    result_image = out.get_image()[:, :, ::-1]
    cv2.imshow(f'Result for {image_path}', result_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def process_video(predictor, video_path):
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        outputs = predictor(frame)
        v = Visualizer(frame[:, :, ::-1], MetadataCatalog.get(predictor.cfg.DATASETS.TRAIN[0]), scale=1.2)
        out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
        
        # Display the frame
        result_frame = out.get_image()[:, :, ::-1]
        cv2.imshow(f'Result for a frame in {video_path}', result_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

def process_folder(predictor, folder_path):
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if os.path.isfile(file_path):
            if file_path.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff')):
                process_image(predictor, file_path)
            elif file_path.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
                process_video(predictor, file_path)

# Example usage
config_file = "C:/Users/Daanish Mittal/OneDrive/Desktop/Tank_align/tank_alignment/ramp_edge_combined/tank-and-ramp/mask_rcnn_R_101_FPN_3x/2024-07-01-16-00-38/config.yaml"
weights_file = "C:/Users/Daanish Mittal/OneDrive/Desktop/Tank_align/tank_alignment/ramp_edge_combined/tank-and-ramp/mask_rcnn_R_101_FPN_3x/2024-07-01-16-00-38/model_final.pth"
input_folder = "C:/Users/Daanish Mittal/OneDrive/Desktop/Tank_align/tank_alignment/ramp_edge_combined/tank-and-ramp-1/test"

predictor = load_model(config_file, weights_file)
process_folder(predictor, input_folder)
